In [1]:
from typing import List, Tuple
import sys
import math
import torch
import pandas
import numpy
from torch_geometric.data import Data
sys.path.append("D:\orguel_ml_library")
from orguel_ml import Graph as BaseGraph

class Graph(BaseGraph):
    __edge_attributes = {
                            "parallel": 0,
                            "colinear": 0,
                            "perpendicular_distance": 0,
                            "overlap_ratio": 0,
                            "point_intersection": 0,
                            "segment_intersection": 0,
                            "angle_difference": 0
                        }
    def __init__(self, dataframe):
        self.__dataframe = dataframe
        
        coordinates_x = dataframe[['start_x', 'end_x']]
        coordinates_y = dataframe[['start_y', 'end_y']]
        
        normalizedCoordinates = dataframe[['start_x', 'start_y', 'end_x', 'end_y']].copy()
        normalizedCoordinates[['start_x', 'end_x']] = (normalizedCoordinates[['start_x', 'end_x']] - coordinates_x.values.mean()) / coordinates_x.values.std()
        normalizedCoordinates[['start_y', 'end_y']] = (normalizedCoordinates[['start_y', 'end_y']] - coordinates_y.values.mean()) / coordinates_y.values.std()
        
        normalizedAngles = numpy.column_stack((numpy.sin(dataframe['angle']), numpy.cos(dataframe['angle'])))
        normalizedLengths = dataframe[['length']] / 176.53671467337412
        
        self.nodesAttributes = numpy.hstack([normalizedCoordinates.values, normalizedAngles, normalizedLengths.values])
        self.edges: List[Tuple[int, int]] = []
        self.edgesAttributes: List[List[float]] = []

def CreateGraph(dataframe):
    # Calculate length, angle, and offset
    def Length(row):
        return ((row["end_x"] - row["start_x"])**2 + (row["end_y"] - row["start_y"])**2) ** 0.5
    
    def Angle(row):
        return math.atan2(row["end_y"] - row["start_y"], row["end_x"] - row["start_x"]) % math.pi
    
    def Offset(row):
        nlen = math.hypot(row["start_y"] - row["end_y"], row["end_x"] - row["start_x"])
        if nlen < 1e-12: return 0.0
        return abs(row["start_x"] * ((row["start_y"] - row["end_y"]) / nlen) +
                   row["start_y"] * ((row["end_x"] - row["start_x"]) / nlen))
    
    dataframe["length"] = dataframe.apply(Length, axis=1)
    dataframe["angle"] = dataframe.apply(Angle, axis=1)
    dataframe["offset"] = dataframe.apply(Offset, axis=1)
    
    graph = Graph(dataframe)
    graph.ParallelDetection()
    graph.IntersectionDetection()
    
    graph.nodesAttributes = torch.tensor(graph.nodesAttributes, dtype=torch.float)
    graph.edges = torch.tensor(graph.edges, dtype=torch.long).t().contiguous()
    graph.edgesAttributes = torch.tensor(graph.edgesAttributes, dtype=torch.float)
    
    return Data(x=graph.nodesAttributes, edge_index=graph.edges, edge_attr=graph.edgesAttributes, y=None)

In [2]:
 # ✅ Step 1: Receive CSV string from AutoCAD
csv_file = "C:\\Users\\Rafael\\Desktop\\text_recognition\\database.csv"

dataframe = pandas.read_csv(csv_file, header=None)
dataframe.columns = ['handle', 'start_x', 'start_y', 'end_x', 'end_y']

for col in ['circle','arc','radius','start_angle','end_angle','layer']:
    dataframe[col] = 0

In [ ]:
# ✅ Step 2: Convert CSV string into a graph
graph: Data = CreateGraph(dataframe)  # Convert AutoCAD data into a graph

In [4]:
import sys
from orguel_ml.ocr import OCRNetwork

# ✅ Step 3: Load trained GNN model
model = OCRNetwork()
model.load_state_dict(torch.load(r"C:\Users\Rafael\Desktop\text_recognition\OCRNetwork.pt", map_location=torch.device('cpu')))
model.eval()

C:\Users\Rafael\AppData\Local\Temp\ipykernel_19904\2248256639.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"C:\Users\Rafael\Desktop\

OCRNetwork(
  (embedding_layer): Sequential(
    (0): Linear(in_features=7, out_features=32, bias=True)
    (1): ReLU()
    (2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (3): Linear(in_features=32, out_features=64, bias=True)
    (4): ReLU()
    (5): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  )
  (conv1): NNConv(64, 128, aggr=add, nn=Sequential(
    (0): Linear(in_features=7, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=8192, bias=True)
  ))
  (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (conv2): NNConv(128, 32, aggr=add, nn=Sequential(
    (0): Linear(in_features=7, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=4096, bias=True)
  ))
  (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (ou

In [5]:
# Run inference
with torch.no_grad():
    output = model(graph)
    char_pred = output["character_logits"].argmax(dim=1).tolist()
    font_pred = output["font_logits"].argmax(dim=1).tolist()
    height, rotation, insertion_x, insertion_y = output["regression_values"].T.tolist()

In [ ]:
# Print results
for idx, handle in enumerate(dataframe['handle']):
    print(f"Line {handle} → char={char_pred[idx]}, font={font_pred[idx]}, height={height[idx]:.2f}, rot={rotation[idx]:.2f}, ins=({insertion_x[idx]:.2f}, {insertion_y[idx]:.2f})")

In [12]:
from orguel_ml.ocr import character_set

character_set["characters"]["encoding"]

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25,
 'a': 26,
 'b': 27,
 'c': 28,
 'd': 29,
 'e': 30,
 'f': 31,
 'g': 32,
 'h': 33,
 'i': 34,
 'j': 35,
 'k': 36,
 'l': 37,
 'm': 38,
 'n': 39,
 'o': 40,
 'p': 41,
 'q': 42,
 'r': 43,
 's': 44,
 't': 45,
 'u': 46,
 'v': 47,
 'w': 48,
 'x': 49,
 'y': 50,
 'z': 51,
 '0': 52,
 '1': 53,
 '2': 54,
 '3': 55,
 '4': 56,
 '5': 57,
 '6': 58,
 '7': 59,
 '8': 60,
 '9': 61,
 '!': 62,
 '"': 63,
 '#': 64,
 '$': 65,
 '%': 66,
 '&': 67,
 "'": 68,
 '*': 69,
 '+': 70,
 ',': 71,
 '-': 72,
 '.': 73,
 '/': 74,
 ':': 75,
 ';': 76,
 '=': 77,
 '?': 78,
 '@': 79,
 '\\': 80,
 '_': 81,
 '`': 82,
 '|': 83,
 '~': 84,
 '(': 85,
 ')': 86,
 '[': 87,
 ']': 88,
 '{': 89,
 '}': 90,
 '<': 91,
 '>': 92,
 'À': 93,
 'Á': 94,
 'Â': 95,
 'Ã': 96,
 'Ç': 97,
 'É': 98,
 'Ê': 99,
 'Í': 100